<a href="https://colab.research.google.com/github/swetzel1/introduction_to_ml/blob/main/hw5/hw5_p3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd

import torch
from torch.autograd import Variable
torch.set_printoptions(edgeitems=2, linewidth=75)
import torch.optim as optim

import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import and preprocess housing dataset
#read data from csv file
file_path = '/content/drive/My Drive/Intro_to_ML/Datasets/Housing.csv'
df = pd.read_csv(file_path)

from IPython.display import display
#display(df)

In [ ]:
#select required parameters
varlist =  ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea']

df = df[varlist]
#df.head()

In [5]:
# List of variables to map
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

# Defining the map function
def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

# Applying the function to the housing list
df[varlist] = df[varlist].apply(binary_map)
#df.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking'] # don't normalize output

df[num_vars] = scaler.fit_transform(df[num_vars])
df.head()

In [7]:
#shuffle and prepare split
n_samples = df.shape[0]
n_val = int(0.2 * n_samples) # 20/80 split

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]


#split training and validation set
y = df.pop('price')
y = y.to_numpy()
x = df.to_numpy()

#convert numpy to tensor
x_train = torch.tensor(x[train_indices], dtype=torch.float32)
y_train = torch.tensor(y[train_indices], dtype=torch.float32)
x_val = torch.tensor(x[val_indices], dtype=torch.float32)
y_val = torch.tensor(y[val_indices], dtype=torch.float32)



In [8]:
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out

In [15]:
#try different learning rates

inputDim = 11          # dim of x
outputDim = 1         # dim of y
epochs = 5001

model = linearRegression(inputDim, outputDim)
criterion = torch.nn.MSELoss()


training_loss_history = []
validation_loss_history = []

#for lr in [0.0002, 0.001, 0.005, 0.01]: #learning rates for SGD
for lr in [0.001, 0.01, 0.1, 0.5]: #learning rates for Adam
#for lr in [0.01]:


  print('Learning rate:', lr)


  #optimizer = torch.optim.SGD(model.parameters(), lr=lr)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)

  for epoch in range(epochs):


    train_in = Variable(x_train)
    train_label = Variable(y_train)
    train_label = train_label.unsqueeze(1)

    val_in = Variable(x_val)
    val_label = Variable(y_val)
    val_label = val_label.unsqueeze(1)


    optimizer.zero_grad() #important
    train_pred = model(train_in)#predict
    loss = criterion(train_pred, train_label)#get train loss

    with torch.no_grad():
      val_pred = model(val_in)
      val_loss = criterion(val_pred, val_label)

    loss.backward()
    optimizer.step()


    #store loss and print
    training_loss_history.append(loss.item())#store loss
    validation_loss_history.append(val_loss.item())

    if epoch % 500 == 0: #report loss every 500 epochs
      print('Epoch %d, Training Loss %f, Validation Loss: %f' % (epoch, float(loss), float(val_loss)))



  #print('\n')
  #plt.plot(training_loss_history, label="Training loss")
  #plt.plot(validation_loss_history, label="Validation loss")
  #plt.xlabel("epochs")
  #plt.ylabel("MSE")
  #plt.legend()
  #plt.title(lr)
  #plt.show()

  print('\n')
  training_loss_history.clear()
  validation_loss_history.clear()



Learning rate: 0.001
Epoch 0, Training Loss 26871444013056.000000, Validation Loss: 23583399084032.000000
Epoch 500, Training Loss 26871420944384.000000, Validation Loss: 23583382306816.000000
Epoch 1000, Training Loss 26871404167168.000000, Validation Loss: 23583365529600.000000
Epoch 1500, Training Loss 26871383195648.000000, Validation Loss: 23583348752384.000000
Epoch 2000, Training Loss 26871362224128.000000, Validation Loss: 23583329878016.000000
Epoch 2500, Training Loss 26871339155456.000000, Validation Loss: 23583315197952.000000
Epoch 3000, Training Loss 26871320281088.000000, Validation Loss: 23583298420736.000000
Epoch 3500, Training Loss 26871297212416.000000, Validation Loss: 23583281643520.000000
Epoch 4000, Training Loss 26871282532352.000000, Validation Loss: 23583266963456.000000
Epoch 4500, Training Loss 26871261560832.000000, Validation Loss: 23583245991936.000000
Epoch 5000, Training Loss 26871238492160.000000, Validation Loss: 23583231311872.000000


Learning rate